In [5]:
#from ..quantumsymmetry import Encoding
#from ..quantumsymmetry.qiskit_converter import get_num_particles_spin_orbitalsf
from ..quantumsymmetry import Encoding
from qiskit.providers.basic_provider import BasicProvider
from qiskit_algorithms.optimizers import SLSQP
from qiskit.primitives import Estimator
from qiskit_algorithms import VQE
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper, QubitMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

from pyscf import gto, scf
from numpy import isclose

def UCC_JW_VQE(atom, basis, charge = 0, spin = 0):
    driver = PySCFDriver(
        atom= atom,
        unit=DistanceUnit.ANGSTROM,
        charge=charge,
        spin=spin,
        basis=basis)
    
    problem = driver.run()

    mapper = JordanWignerMapper()

    initial_state = HartreeFock(
        num_spatial_orbitals = problem.num_spatial_orbitals,
        num_particles = problem.num_particles,
        qubit_mapper = mapper
        )
    
    ansatz = UCCSD(
        num_spatial_orbitals = problem.num_spatial_orbitals,
        num_particles = problem.num_particles,
        qubit_mapper = mapper,
        initial_state = initial_state
        )

    vqe = VQE(
        estimator = Estimator(),
        ansatz = ansatz,
        optimizer = SLSQP()
   )

    solver = GroundStateEigensolver(mapper, vqe)
    vqe_result = solver.solve(problem)

    energy1 = vqe_result.total_energies[0]
    qubits1 = ansatz.num_qubits
    depth1 = ansatz.decompose().decompose().decompose().decompose().depth()
    
    return energy1

def UCC_JW_VQE(atom, basis, charge = 0, spin = 0):
    driver = PySCFDriver(
        atom= atom,
        unit=DistanceUnit.ANGSTROM,
        charge=charge,
        spin=spin,
        basis=basis)
    
    problem = driver.run()

    mapper = QubitMapper()
    encoding = Encoding(atom = atom, basis = basis, charge = charge, spin = spin)
    mapper.map = encoding.apply_encoding()

    initial_state = HartreeFock(
        num_spatial_orbitals = problem.num_spatial_orbitals,
        num_particles = problem.num_particles,
        qubit_mapper = mapper
        )
    
    ansatz = UCCSD(
        num_spatial_orbitals = problem.num_spatial_orbitals,
        num_particles = problem.num_particles,
        qubit_mapper = mapper,
        initial_state = initial_state
        )

    vqe = VQE(
        estimator = Estimator(),
        ansatz = ansatz,
        optimizer = SLSQP()
   )

    solver = GroundStateEigensolver(mapper, vqe)
    vqe_result = solver.solve(problem)

    energy1 = vqe_result.total_energies[0]
    qubits1 = ansatz.num_qubits
    depth1 = ansatz.decompose().decompose().decompose().decompose().depth()
    
    return energy1

def UCC_SAE_VQE(atom, basis, charge = 0, spin = 0, CAS = None):
    encoding = Encoding(atom = atom, basis = basis, charge = charge, spin = spin, verbose = False, CAS = CAS, output_format = 'qiskit')
    qubit_operator = encoding.hamiltonian
    ansatz = encoding.UCC_circuit
    
    #qubit_operator = reduced_hamiltonian(atom = atom, basis = basis, charge = charge, spin = spin, CAS = CAS, output_format = 'qiskit', verbose = False)
    #ansatz = UCC_SAE_circuit(atom = atom, basis = basis, charge = charge, spin = spin, CAS = CAS)

    backend = BasicProvider().get_backend("statevector_simulator")
    quantum_instance=QuantumInstance(backend=backend)

    initial_point = [0]*ansatz.num_parameters

    vqe = VQE(
        ansatz = ansatz,
        initial_point = initial_point,
            quantum_instance=quantum_instance)

    vqe_result = vqe.compute_minimum_eigenvalue(qubit_operator)

    energy2 = vqe_result.optimal_value
    qubits2 = ansatz.num_qubits
    depth2 = ansatz.decompose().decompose().decompose().decompose().depth()
    
    return energy2

def get_energy_nuc(atom, basis, charge = 0, spin = 0):
    mol = gto.Mole()
    mol.atom = atom
    mol.symmetry = True
    mol.basis = basis
    mol.charge = charge
    mol.spin = spin
    mol.verbose = 0
    mol.build()
    mf = scf.RHF(mol)
    mf.kernel()
    energy_nuc = mf.energy_nuc()
    
    return energy_nuc

def test_UCC_circuit_H3():
    #Parameters for H3+ in the sto-3g basis
    atom = 'H 0 0.377 0; H -0.435 -0.377 0; H 0.435 -0.377 0'
    charge = 1
    basis = 'sto3g'

    energy1 = UCC_JW_VQE(atom = atom, basis = basis, charge = charge)
    energy2 = UCC_SAE_VQE(atom = atom, basis = basis, charge = charge)
    energy_nuc = get_energy_nuc(atom = atom, basis = basis, charge = charge)
    
    assert isclose(energy1, energy2 - energy_nuc)

def test_UCC_circuit_H2O_CAS():
    #Parameters for H2O in the sto-3g basis with CAS(4, 4)
    atom = 'O 0 0 0.1197; H 0 0.7616 -0.4786; H 0 -0.7616 -0.4786'
    basis = 'sto3g'
    CAS = (4, 4)

    energy = UCC_SAE_VQE(atom = atom, basis = basis, CAS = CAS)
    
    assert isclose(energy, -74.97227790592032)

atom = 'H 0 0.377 0; H -0.435 -0.377 0; H 0.435 -0.377 0'
charge = 1
basis = 'sto3g'

UCC_JW_VQE(atom, basis, charge = charge, spin = 0)

ImportError: attempted relative import with no known parent package